### Import  libraries and read data

In [ ]:
# pip install pandas-datareader

### Initial Imports

In [1]:
# from dotenv import load_dotenv
# import alpaca_trade_api as tradeapi
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import numpy as np
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Universal variable

T1=10
T2=5
stock="AMZN"
benchmark="SPY"
yield10y="^TNX"
crypto="BTC-USD"


end = datetime.date.today()
ed=end.strftime('%Y-%m-%d')

data_begin=(end-pd.DateOffset(365*T1))
st=data_begin.strftime('%Y-%m-%d')

visual_begin=end-pd.DateOffset(365*T2)
vst=visual_begin.strftime('%Y-%m-%d')

In [15]:
# Universal Functions

def n_year_ago(n):
    # to get the date of "n years ago from today"
    n_years_ago=(end-pd.DateOffset(365*n)).strftime('%Y-%m-%d')
    return n_years_ago


### Import data from Yahoo Finance

In [3]:
# Import data from 
def import_data(ticker):
    ticker_df = web.DataReader(ticker, 'yahoo', st, ed)
    ticker_close=round(ticker_df[['Close']],2)
    return ticker_close


### Metric Calculation
##### Create function to generate metrics

* 1-year/5-year annualized return 
* 1-year/5-year cumulative return 
* annualized rolling volatility 
* annualized rolling sharpe ratio 


In [5]:
def daily_return(ticker):
    # calculate daily return 
    df=import_data(ticker)
    df['daily_return']=df['Close'].pct_change()
    return df[['daily_return']]

def yr_a_rt(ticker,years):
    # calculate n-year annualized return
    df=import_data(ticker)
    df[f'{years}yr_a_rt']=(1+df['Close'].pct_change(periods=252*years))**(1/years) - 1
    return df[[f'{years}yr_a_rt']]

def yr_c_rt(ticker,years):
    # calculate n-year cum return
    df=import_data(ticker)
    df['daily_return']=df['Close'].pct_change()
    df[f'{years}yr_c_rt'] = (1 + df['daily_return'])[n_year_ago(years):].cumprod() - 1  
    return df[[f'{years}yr_c_rt']]

def annualized_stddv(ticker):
    # calculate annualized rolling standard deviation:
    df=import_data(ticker)
    df['daily_return']=df['Close'].pct_change()
    df['annualizd_volatility']=df['daily_return'].rolling(252).std()*(252**0.5)
    return df[['annualizd_volatility']]

def annualized_sharpe(ticker):
    # calculate annualized rolling sharpe ratio:
    df=import_data(ticker)
    df['daily_return']=df['Close'].pct_change()
    df['annualizd_sharpe']=(252**0.5)*(df['daily_return'].rolling(252).mean()
                                    /df['daily_return'].rolling(252).std())
    return df[['annualizd_sharpe']]
    
def correlation(data):
    correlation = np.log(data/data.shift())
    return correlation.corr()

### Create Function on Plotting:


In [12]:
# format data into proper structure to be used in chart function

def format_data(ticker1,metric1,ticker2,metric2):
    # metric 1 and 2 are two dataframe generated through metric calculation functions
    metric1['Label']=ticker1
    metric2['Label']=ticker2
    c_df=pd.concat([metric1,metric2])
    
    
    c_df.dropna(inplace=True)

    return c_df

In [13]:
# format data into proper structure to be used in chart function
def plot_lines(df,ylable,title,format):
    fig = px.line(df, x=df.index, y=list(df.columns)[0], color=list(df.columns)[1]
                ,labels={
                        list(df.columns)[0]: ylable
                    }
                ,title=title)
    fig.update_layout(yaxis_tickformat = format)
    return fig


In [ ]:
def line_plot(data, label, title, xlabel, ylabel):
    return data.plot(xlabel=xlabel, ylabel=ylabel, figsize=(15,7), label=label)

In [ ]:
def kde_plot(data, label):
    return data.plot(kind='kde', label=label, figsize=(15,7)), plt.legend()

In [ ]:
def histogram(data, label):
    return data.hist(bins=100, label=label, alpha=0.5, figsize=(15,7)), plt.legend()

### Part 1 - Closing Price (use hasnain's code for this part)

For user to have a overview of the historical price movement and volumn 

### Part 2 - Risk and return comparison between selected stock {AMZN} and benchmark {SPY}
direct comparison on the following metrics
* cumulative returns (from 1 year ago, from 5 years ago)
* annualized returns (1-year average, 5-year average) 
* risk (annualized daily return standard deviation)   
* risk-adjusted returns (sharp ratio)

#### Annualized Returns

In [88]:
df2_1=format_data(stock,yr_a_rt(stock,1)[vst:],benchmark,yr_a_rt(benchmark,1)[vst:])
plot_lines(df2_1,'Annualized Return',f'Rolling 1-year Annualized Return: {stock} vs. {benchmark}','0.2%')

In [63]:
df2_2=format_data(stock,yr_a_rt(stock,5)[vst:],benchmark,yr_a_rt(benchmark,5)[vst:])
plot_lines(df2_2,'Annualized Return',f'Rolling 5-year Annualized Return: {stock} vs. {benchmark}','0.2%')

/Users/feiyiyang/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.



#### Cumulative Returns (rolling)

In [64]:
df=format_data(stock,yr_c_rt(stock,1)[vst:],benchmark,yr_c_rt(benchmark,1)[vst:])
plot_lines(df,'Cumulative Return',f'Rolling 1-year Cumulative Return: {stock} vs. {benchmark}','0.2%')

/Users/feiyiyang/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.



In [24]:
df=format_data(stock,yr_c_rt(stock,5)[vst:],benchmark,yr_c_rt(benchmark,5)[vst:])
plot_lines(df,'Cumulative Return',f'Rolling 1-year Annualized Return: {stock} vs. {benchmark}','0.2%')

#### Risk (std.dev)

In [32]:
df=format_data(stock,annualized_stddv(stock)[vst:],benchmark,annualized_stddv(benchmark)[vst:])
plot_lines(df,'Volatility',f'Rolling Annualized Volatility: {stock} vs. {benchmark}','0.2%')

#### Risk-adjusted Return (sharp ratio)
*assuming risk-free rate is 0

In [65]:
df=format_data(stock,annualized_sharpe(stock)[vst:],benchmark,annualized_sharpe(benchmark)[vst:])
plot_lines(df,'Sharpe Ratio',f'Rolling Annualized Sharpe Ratio: {stock} vs. {benchmark}','0.2')

### Part 3 - Correlation and Diversification 

These charts should give user/investor an idea of how the stock (in this case "AMZN") is correlated with the broad market (SPY), with interest rate (10-year Bond Yield), and with crypto price (Bitcoin), thus helping them to make decisions from a portfolio diversification point of view

In [28]:
def data_corr(ticker1,ticker2,years):
    # metric 1 and 2 are two dataframe generated through metric calculation functions

    c_df=pd.concat([daily_return(ticker1)[n_year_ago(years):],daily_return(ticker2)[n_year_ago(years):]],axis=1)
    c_df.dropna(inplace=True)

    return c_df.corr().iloc[0,1]

In [29]:
print(data_corr(stock,crypto,1))
print(data_corr(stock,crypto,5))

0.2657132681633214
0.15811803098713775


In [30]:
def correlation_gauge(years):
    fig = go.Figure()

    fig.add_trace(go.Indicator(
        mode = "gauge+number",
        title = {'text': f"Correlation with {benchmark}"},
        value = data_corr(stock,benchmark,years),
        # delta = {'reference': data_corr(stock,benchmark)},
        gauge = {
            'axis': {'visible': True, 'range':(-1,1)}},
        domain = {'row': 0, 'column': 0}))
    fig.add_trace(go.Indicator(
        mode = "gauge+number",
        title = {'text': f"Correlation with Treasury Yield"},
        value = data_corr(stock,yield10y,years),
        # delta = {'reference': data_corr(stock,yield10y)},
        gauge = {
            'axis': {'visible': True, 'range':(-1,1)}},
        domain = {'row': 0, 'column': 1}))
    fig.add_trace(go.Indicator(
        mode = "gauge+number",
        title = {'text': f"Correlation with Bitcoin"},
        value = data_corr(stock,crypto,years),
        # delta = {'reference': data_corr(stock,crypto)},
        gauge = {
            'axis': {'visible': True, 'range':(-1,1)}},
        domain = {'row': 0, 'column': 2}))
    fig.update_layout(
        grid = {'rows': 1, 'columns': 3, 'pattern': "independent"},
        template = {'data' : {'indicator': [{
            'delta' : {'reference': 1}}]
        }})
    return fig

#### (short-term) Correlation during the past 1 year

In [31]:
correlation_gauge(1)

##### (long-term) Correlation based on 5 years of historical price

In [32]:
correlation_gauge(5)